#### LangChain文本向量化存储

##### 配置文件准备

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from openai.env file
load_dotenv(".env")

# Read the OPENAI_API_KEY from the environment
api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

##### 检查依赖安装

In [ ]:
%pip show openai
%pip show langchain
%pip show langchain-openai

##### 词嵌入:将一组文本转换为向量表示

In [ ]:
# 使用OpenAI词嵌入模型
from langchain_community.embeddings import OpenAIEmbeddings

# 1.准备文本数据
texts = [
    "你好",
    "你好啊",
    "你叫什么名字?",
    "我叫王大锤",
    "很高兴认识你大锤",
]

# 2.创建词嵌入模型
e_model = OpenAIEmbeddings()

# 生成词嵌入，返回包含对应词嵌入向量的列表
ebeddings = e_model.embed_documents(texts)

# 返回的 ebeddings 变量是一个列表，其中每个元素都是一个 NumPy 数组，表示对应文本的词嵌入向量。
ebeddings

##### 向量化的应用场景：语义搜索

In [ ]:
# 将文本转换为向量后，可以计算向量之间的相似度，从而找到语义上相似的文本。
embedded_query = e_model.embed_query("这段对话中提到了什么名字?")

# 获取 embedded_query 数组的前五个元素
embedded_query[:5]

##### 嵌入向量缓存

In [ ]:
# 带有缓存功能的嵌入模型
from langchain.embeddings import CacheBackedEmbeddings
# 用于在本地文件系统中存储键值对，用于缓存数据。
from langchain.storage import LocalFileStore
# 用于从文本文件中加载文档
from langchain_community.document_loaders import TextLoader
# 用于按字符分割文本
from langchain.text_splitter import CharacterTextSplitter
# 使用 OpenAI 的嵌入模型
from langchain_community.embeddings import OpenAIEmbeddings

# 1.创建词嵌入模型
u_embeddings = OpenAIEmbeddings()
# 2.创建本地文件存储实例
fs = LocalFileStore("./cache/")
# 3.创建缓存嵌入模型实例
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    u_embeddings,
    fs,
    namespace=u_embeddings.model, # 这将缓存命名空间设置为原始嵌入模型的名称，以避免不同模型之间的缓存冲突。
)

# fs.yield_keys(): 这个方法生成一个包含缓存中所有键的迭代器。
# list(): 这个函数将迭代器转换为列表，以便打印缓存中的所有键。
list(fs.yield_keys())

In [ ]:
# 加载文档，切分文档

raw_documents = TextLoader("./source/letter.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
%pip install faiss-cup

In [ ]:
# 将切分文档向量化病存储在缓存中
# 导入FAISS 向量数据库
from langchain_community.vectorstores import FAISS

# %timeit 是 Jupyter Notebook 的魔术命令
# 测量了从文档列表创建 FAISS 索引的耗时
%timeit -r  1 -n 1 db = FAISS.from_documents(documents, cached_embeddings)

In [ ]:
# 查看缓存中的键
list(fs.yield_keys())